# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [3]:
Summaries_file = 'recognition_Summaries.pkl.bz2'
Abstracts_file = 'recognition_Abstracts.pkl.bz2'

In [4]:

import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [5]:
Summaries[11802866]

paper(title='Surveying the seen: 100 years of British vision.', authors=['Wade NJ', 'Bruce V'], year=2001, doi='')

In [6]:
Abstracts[11802866]

"Perceptual phenomena and their interpretations have fashioned the course of psychology. This article surveys how theories of visual perception and methodologies have developed during the lifetime of the British Psychological Society. The experimental study of vision was instigated by British natural philosophers in the early nineteenth century but this impetus was not maintained thereafter. Not until the 1930s and 1940s did research on perception resume in earnest within British universities. The adoption of concepts (such as schema) potentially grounded in neural organization, particularly by Bartlett and Craik, accelerated experimental, theoretical and applied vision research. From mid-century the influence of information processing models of perception became increasingly dominant, and they were often integrated with the rapidly expanding understanding of neurophysiological underpinnings. The epitome of these developments was Marr's model of vision which, in our view, marked the st

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [7]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on the list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Recognition of the positive effects of music and dance for mental health.")))





['recognition', 'of', 'the', 'positive', 'effects', 'of', 'music', 'and', 'dance', 'for', 'mental', 'health.']


In [8]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(36359370, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [9]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a minute:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'guitar':

In [10]:
print(inverted_index['guitar'])

[2343768, 17258164, 20919784, 21983182, 28479473, 32574363, 33114599, 35957448, 37707802, 38076476]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'guitar':

In [9]:
query_word = 'guitar'
for i in inverted_index[query_word]:


----------

# Tasks

**Your name:** Jaspreet Singh


### Task 1

Implement the function `merge_sorted_and` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already (e.g. [3,5,8,11] and [5,7,8,9,11,12]), and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [101]:
# your code here

#assuming we have two sorted posting lists as input

def merge_sorted_and(sortedList1, sortedList2):
    mergeSortedList= []
    index1 = 0
    index2 = 0
    size1 = len(sortedList1)
    size2 = len(sortedList2)
    seen_all_index = False
    while seen_all_index == False:
        if((size1 != index1) and (size2 != index2) ):
            if sortedList1[index1] == sortedList2[index2]:
                mergeSortedList.append(sortedList1[index1])
                index1 +=1
                index2 +=1
                continue
            if(sortedList1[index1]>sortedList2[index2]):
                index2 +=1
                continue
            if(sortedList2[index2]>sortedList1[index1]):
                index1 +=1
                continue
        else:
            seen_all_index = True
    return mergeSortedList
            

In [102]:
firstSortedList = [2,4,8,16,32, 43, 64,128,130]
secondSortedList = [1,2,3,5,8,13,21,43,128]

print(merge_sorted_and(firstSortedList,secondSortedList))

[2, 8, 43, 128]


### Task 2

Similarly as above, implement the function `merge_sorted_or` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [99]:
# Your code here


def merge_sorted_or(sortedList1, sortedList2):
    mergeSortedList= []
    index1 = 0
    index2 = 0
    size1 = len(sortedList1)
    size2 = len(sortedList2)
    seen_all_index = False
    while seen_all_index == False:
        if((size1 != index1) and (size2 != index2) ):
            if sortedList1[index1] == sortedList2[index2]:
                mergeSortedList.append(sortedList1[index1])
                index1 +=1
                index2 +=1
                continue
            if(sortedList1[index1]>sortedList2[index2]):
                mergeSortedList.append(sortedList2[index2])
                index2 +=1
                continue
            if(sortedList2[index2]>sortedList1[index1]):
                mergeSortedList.append(sortedList1[index1])
                index1 +=1
                continue
        
        else:
            if(size1 != index1):
                mergeSortedList.append(sortedList1[index1])
                index1+=1
                continue
            if(size2 != index2):
                mergeSortedList.append(sortedList2[index2])
                index2+=1
                continue
            else:
                seen_all_index = True
    return mergeSortedList
            

In [100]:
firstSortedList = [2,4,8,16,32, 43, 64,128,130,192,201]
secondSortedList = [1,2,3,5,8,13,21,43,45,49,55,65]



print(merge_sorted_or(firstSortedList,secondSortedList))

[1, 2, 3, 4, 5, 8, 13, 16, 21, 32, 43, 45, 49, 55, 64, 65, 128, 130, 192, 201]


### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `merge_and` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [97]:
# Your code here
def and_query(input_query):
    preproccesedTokens = preprocess(tokenize(input_query))
    
    posting_lists = []

    for token in preproccesedTokens:
        if(token in inverted_index):
            posting_lists.append(inverted_index[token])
            
    currentAndQuery = posting_lists[0]
    for i in range(1, len(posting_lists)):
        currentAndQuery = merge_sorted_and(currentAndQuery, posting_lists[i])
        
    return currentAndQuery
        
    
    


In [98]:
print(and_query("music guitar")) 

[32574363, 33114599, 38076476]


### Task 4

Construct another function called `query_or` that works in the same way as `query_and` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `merge_sorted_or` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [95]:
# Your code here
def query_or(input_query):
    preproccesedTokens = preprocess(tokenize(input_query))
    
    posting_lists = []

    for token in preproccesedTokens:
        if(token in inverted_index):
            posting_lists.append(inverted_index[token])
            
    currentQueryOr = posting_lists[0]
    for i in range(1, len(posting_lists)):
        currentQueryOr = merge_sorted_or(currentQueryOr, posting_lists[i])
        
    return currentQueryOr
        

In [96]:
# Your code here
print(query_or("apple juice")) 


[65843, 1123708, 1610160, 1777964, 1807757, 2004265, 2329375, 2501961, 2637575, 2683821, 3211683, 3391940, 3835034, 3970548, 4017559, 6890395, 7002504, 7175510, 7646945, 8019865, 8019866, 8020854, 8217304, 8324639, 8466663, 8901343, 8992485, 9494655, 10415933, 10511326, 10552763, 10688276, 11079693, 11113263, 11118438, 11132035, 11299017, 11382326, 11448741, 11519812, 11824606, 11941428, 12120229, 12561204, 12632242, 12744294, 12890998, 12956512, 14574449, 14669896, 15137839, 15446713, 15553641, 15612789, 15925307, 15964491, 16293967, 16596981, 16608236, 16737285, 16881705, 16926543, 17068604, 17189596, 17438537, 17488020, 17554908, 18048008, 18055622, 18355960, 18810293, 19052540, 19067467, 19154550, 19489112, 19528028, 19609881, 19801343, 19907094, 19933774, 20047654, 20140274, 20146077, 20442968, 20477470, 20718565, 20967459, 21435473, 21548650, 21742403, 21889151, 22132537, 22255015, 22349025, 22399295, 22481163, 22563122, 22564930, 22725621, 22791258, 22918534, 23009695, 23035847,

### Task 5

Why does `query_sorted_and('music guitar')` not return our example paper 28450846, even though it mentions these terms in the title and abstract? (You do not have to implement anything to fix this yet!)

**Answer:** In the example paper 28450846, the word "guitar" is written as "guitar," with a comma following it. When we tokenize this, it appears as "guitar," rather than "guitar" alone. To handle this accurately, we need to adjust the tokenization function, which currently splits by spaces. It would be better to remove punctuation marks like ".", ",", ";", and ":" during tokenization. This issue is why the example paper isn’t appearing in our posting list results.








# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.